In [ ]:
######### this one works!!!!
#!/usr/bin/env python3
import json
import heapq
from geopy.distance import geodesic

class LaneNode:
    def __init__(self, path_id, start, end, center):
        self.path_id = path_id
        self.start = tuple(start)
        self.end = tuple(end)
        self.center = center
        self.g = float('inf')  # Initialize with infinity for A*
        self.h = 0
        self.f = 0
        self.parent = None

    def __eq__(self, other):
        return self.path_id == other.path_id

    def __lt__(self, other):
        return self.f < other.f

def heuristic(a, b):
    return geodesic(a.end, b.start).meters

def load_lane_graph(lanes_file):
    with open(lanes_file, 'r') as f:
        lanes_data = json.load(f)["lanes"]

    lanes = {}
    graph = {}

    for lane in lanes_data:
        path_id = lane["path_id"]
        start = lane["start"]
        end = lane["end"]
        center = lane["center"]
        connected_to = lane["connected_to"]

        lane_node = LaneNode(path_id, start, end, center)
        lanes[path_id] = lane_node

        # Filter out invalid connections (e.g., -1)
        valid_connections = [conn for conn in connected_to if conn != -1]
        for connection in valid_connections:
            graph.setdefault(path_id, []).append(connection)
            graph.setdefault(connection, []).append(path_id)  # Ensure reverse connection


    return lanes, graph

def find_closest_lane_node(lanes, gps_point):
    closest_lane = None
    min_distance = float('inf')

    for lane in lanes.values():
        for point in lane.center:
            distance = geodesic(gps_point, point).meters
            if distance < min_distance:
                min_distance = distance
                closest_lane = lane


    return closest_lane

def a_star_lane_level(graph, lanes, start_lane, goal_lane):
    open_list = []
    closed_list = set()

    # Initialize the start node
    start_lane.g = 0
    start_lane.f = heuristic(start_lane, goal_lane)

    heapq.heappush(open_list, start_lane)

    while open_list:
        current_node = heapq.heappop(open_list)
        closed_list.add(current_node.path_id)

        # Check if we've reached the goal
        if current_node == goal_lane:
            path = []
            while current_node:
                path.append(current_node)
                current_node = current_node.parent
            return path[::-1]

        # Explore neighbors
        for neighbor_id in graph.get(current_node.path_id, []):
            if neighbor_id in closed_list:
                continue
            if neighbor_id not in lanes:
                print(f"Warning: Lane ID {neighbor_id} not found in lanes. Skipping.")
                continue

            neighbor_node = lanes[neighbor_id]
            tentative_g = current_node.g + heuristic(current_node, neighbor_node)

            if tentative_g < neighbor_node.g or neighbor_node not in open_list:
                neighbor_node.g = tentative_g
                neighbor_node.h = heuristic(neighbor_node, goal_lane)
                neighbor_node.f = neighbor_node.g + neighbor_node.h
                neighbor_node.parent = current_node

                if neighbor_node not in open_list:
                    heapq.heappush(open_list, neighbor_node)

    return None

def extract_centerline_path(lane_path):
    complete_centerline = []
    for lane in lane_path:
        complete_centerline.extend(lane.center)
    return complete_centerline

if __name__ == "__main__":
    lanes_file = "/home/byron/catkin_ws/src/a_star/src/lanes.json"  # Update with the correct path to lanes.json

    # Load lanes and graph
    lanes, graph = load_lane_graph(lanes_file)

    # Define GPS coordinates for start and goal
    start_gps = (-79.77031442877464, 36.074098649937355)  # Example coordinates
    goal_gps = (-79.769405611563, 36.075825380043085)  # Example coordinates


    # Find closest lanes to the start and goal points
    start_lane = find_closest_lane_node(lanes, start_gps)
    goal_lane = find_closest_lane_node(lanes, goal_gps)

    if not start_lane or not goal_lane:
        print("Error: Could not find closest lanes for the given GPS points.")
    else:
        # Perform A* lane-level planning
        lane_path = a_star_lane_level(graph, lanes, start_lane, goal_lane)

        if lane_path:
            centerline_path = extract_centerline_path(lane_path)
            print("Lane-level Path Centerline:")
            for point in centerline_path:
                print(point)
        else:
            print("No path found.")




Lane-level Path Centerline:
[-79.7704990580678, 36.074017929794756]
[-79.77048698812723, 36.074018200784046]
[-79.77047357708216, 36.074017929794756]
[-79.77046251296997, 36.074018742762576]
[-79.77045278996229, 36.07401901375184]
[-79.77044038474558, 36.07402036869816]
[-79.77042730897665, 36.07402307859074]
[-79.77041523903607, 36.07402714342945]
[-79.770399145782, 36.074033376181724]
[-79.77039076387882, 36.07403744101989]
[-79.77038506418465, 36.07404069289029]
[-79.7703729942441, 36.074048551576496]
[-79.77036461234093, 36.07405559729448]
[-79.77035488933323, 36.07406426894654]
[-79.77034583687782, 36.07407483752119]
[-79.77033946663141, 36.07408296719304]
[-79.77033242583275, 36.07409163884208]
[-79.77032873779535, 36.074101665435094]
[-79.77032739669085, 36.07411385993837]
[-79.77032739669085, 36.07411385993837]
[-79.77032739669085, 36.07412632542868]
[-79.77032706141472, 36.07413689399498]
[-79.77032672613859, 36.07414990145926]
[-79.77032639086246, 36.0741629089214]
[-79.77032

In [29]:
import folium

def generate_map(start_gps, goal_gps, centerline_path, output_file="a3_compy.html"):
    """
    Generates an HTML file visualizing the A* path with start and goal points.
    
    Parameters:
        start_gps (list): [longitude, latitude] of the start point.
        goal_gps (list): [longitude, latitude] of the goal point.
        centerline_path (list): List of [longitude, latitude] pairs representing the A* path.
        output_file (str): Name of the HTML file to save the map.
    """
    try:
        # Convert [longitude, latitude] to [latitude, longitude] for Folium
        converted_path = [[pt[1], pt[0]] for pt in centerline_path]
        converted_start = [start_gps[1], start_gps[0]]
        converted_goal = [goal_gps[1], goal_gps[0]]
        
        # Determine map center as the start point
        map_center = converted_start
        
        # Create the map
        m = folium.Map(location=map_center, zoom_start=16)
        
        # Add path to map
        folium.PolyLine(converted_path, color="blue", weight=4, opacity=0.8, popup="A* Path").add_to(m)
        
        # Add start and goal markers
        folium.Marker(converted_start, popup="Start", icon=folium.Icon(color="green")).add_to(m)
        folium.Marker(converted_goal, popup="Goal", icon=folium.Icon(color="red")).add_to(m)
        
        # Add layer control for interactivity
        folium.LayerControl().add_to(m)
        
        # Save the map as an HTML file
        m.save(output_file)
        print(f"Map successfully saved to {output_file}")
        
    except Exception as e:
        print(f"Error generating map: {e}")

if __name__ == "__main__":
    # # Example data for testing
    # start_gps = [-79.77260794490576, 36.07420816403331]  # [longitude, latitude]
    # goal_gps = [-79.7712742164731, 36.07397402952052]  # [longitude, latitude]
    # centerline_path = [
    #     [-79.77260794490576, 36.07420816403331],
    #     [-79.77234523423423, 36.07410011232323],
    #     [-79.77170011112233, 36.07398778899012],
    #     [-79.7712742164731, 36.07397402952052]
    # ]
    
    # Run the map generation function
    generate_map(start_gps, goal_gps, centerline_path)


Map successfully saved to a3_compy.html
